In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# input and output data
train_x = [[0., 0.], [0., 1.], [1., 0.], [1., 1.], [0., 0.], [0., 1.], [1., 0.], [1., 1.]]  # XOR input
train_y = [[0.], [1.], [1.], [0.], [0.], [1.], [1.], [0.]]  # XOR output

test_x = [[0., 0.], [0., 1.], [1., 0.], [1., 1.], [0., 0.], [0., 1.], [1., 0.], [1., 1.]]  # XOR input
test_y = [[0.], [1.], [1.], [0.], [0.], [1.], [1.], [0.]]  # XOR output

# placeholder
_x = tf.placeholder(tf.float32, shape=[None, 2], name="placeholder_x")
_y = tf.placeholder(tf.float32, shape=[None, 1], name="placeholder_y")
_d = tf.placeholder(tf.float32, name="placeholder_dropout")

# neurons
hidden_nodes = 3

# Store layers weight and bias
w = {
    'h1': tf.Variable(tf.truncated_normal([2, hidden_nodes], stddev=0.1), name="h1_w"),
    'h2': tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=0.1), name="h2_w"),
    'h3': tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=0.1), name="h3_w"),
    'out': tf.Variable(tf.truncated_normal([hidden_nodes, 1], stddev=0.1), name="out_w")
}
b = {
    'b1': tf.Variable(tf.constant(0.1, shape=[hidden_nodes]), name="h1_b"),
    'b2': tf.Variable(tf.constant(0.1, shape=[hidden_nodes]), name="h2_b"),
    'b3': tf.Variable(tf.constant(0.1, shape=[hidden_nodes]), name="h3_b"),
    'out': tf.Variable(tf.constant(0.1, shape=[1]), name="out_b")
}


################
# hidden layer #
################
layer_1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(_x, w['h1']), b['b1'])), _d) # dropout
layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, w['h2']), b['b2']))
#layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, w['h3']), b['b3']))

################
# output layer #
################
output = tf.sigmoid(tf.matmul(layer_2, w['out']))

############
# learning #
############
cross_entropy = -(_y * tf.log(output) + (1 - _y) * tf.log(1 - output))
# cross_entropy = tf.square(_y - output)  # simpler, but also works

loss = tf.reduce_mean(cross_entropy)  # mean the cross_entropy
optimizer = tf.train.AdamOptimizer(0.01)  # take a gradient descent for optimizing with a "stepsize" of 0.1
train = optimizer.minimize(loss)  # let the optimizer train

tf.scalar_summary("loss", loss)
summary = tf.merge_all_summaries()

In [3]:
####################
# initialize graph #
####################
init = tf.initialize_all_variables()

sess = tf.Session()  # create the session and therefore the graph
sess.run(init)  # initialize all variables 
best_loss = float("inf")
best_accuracy = 0.0
writer = tf.train.SummaryWriter('graph/xor', graph=sess.graph)
saver = tf.train.Saver()
#####################
# train the network #
#####################
print("start")
for epoch in xrange(0, 2001):
    # run the training operation
    cvalues = sess.run([train, loss, w['h1'], b['b1'], w['h2'], b['b2'], w['out']],
                       feed_dict={_x: train_x, _y: train_y, _d: 0.5})
  
    # print some debug stuff
    if epoch % 200 == 0:
        print("step: {:>3}".format(epoch))
        print("loss: {}".format(cvalues[1]))
        #print("b1: {}".format(cvalues[3]))
        #print("W_hidden: {}".format(cvalues[2]))
        #print("W_output: {}".format(cvalues[4]))
        summary_str = sess.run(summary, feed_dict={_x: train_x, _y: train_y, _d: 1.0})
        writer.add_summary(summary_str, epoch)
      
    # stop Over-Fitting
    test_loss = sess.run(loss, feed_dict={_x: train_x, _y: train_y, _d: 1.0})
    if test_loss < best_loss:
        best_loss = test_loss
        saver.save(sess, "graph/tf_test_xor.ckpt")
        # best
        print("best")
        print(" - input: {} | output: {}".format(test_x[0], sess.run(output, feed_dict={_x: [test_x[0]], _d: 1.0})))
        print(" - input: {} | output: {}".format(test_x[1], sess.run(output, feed_dict={_x: [test_x[1]], _d: 1.0})))
        print(" - input: {} | output: {}".format(test_x[2], sess.run(output, feed_dict={_x: [test_x[2]], _d: 1.0})))
        print(" - input: {} | output: {}".format(test_x[3], sess.run(output, feed_dict={_x: [test_x[3]], _d: 1.0})))


print("end")

saver.restore(sess, "graph/tf_test_xor.ckpt")
print("input: {} | output: {}".format(test_x[0], sess.run(output, feed_dict={_x: [test_x[0]], _d: 1.0})))
print("input: {} | output: {}".format(test_x[1], sess.run(output, feed_dict={_x: [test_x[1]], _d: 1.0})))
print("input: {} | output: {}".format(test_x[2], sess.run(output, feed_dict={_x: [test_x[2]], _d: 1.0})))
print("input: {} | output: {}".format(test_x[3], sess.run(output, feed_dict={_x: [test_x[3]], _d: 1.0})))

start
step:   0
loss: 0.692052781582
best
 - input: [0.0, 0.0] | output: [[ 0.5053969]]
 - input: [0.0, 1.0] | output: [[ 0.50383478]]
 - input: [1.0, 0.0] | output: [[ 0.50597513]]
 - input: [1.0, 1.0] | output: [[ 0.5043931]]
best
 - input: [0.0, 0.0] | output: [[ 0.50506783]]
 - input: [0.0, 1.0] | output: [[ 0.503609]]
 - input: [1.0, 0.0] | output: [[ 0.50549483]]
 - input: [1.0, 1.0] | output: [[ 0.50401932]]
best
 - input: [0.0, 0.0] | output: [[ 0.50479376]]
 - input: [0.0, 1.0] | output: [[ 0.50334555]]
 - input: [1.0, 0.0] | output: [[ 0.50519639]]
 - input: [1.0, 1.0] | output: [[ 0.50373161]]
best
 - input: [0.0, 0.0] | output: [[ 0.50444275]]
 - input: [0.0, 1.0] | output: [[ 0.50319088]]
 - input: [1.0, 0.0] | output: [[ 0.50478554]]
 - input: [1.0, 1.0] | output: [[ 0.50352132]]
best
 - input: [0.0, 0.0] | output: [[ 0.50387186]]
 - input: [0.0, 1.0] | output: [[ 0.50283659]]
 - input: [1.0, 0.0] | output: [[ 0.50413549]]
 - input: [1.0, 1.0] | output: [[ 0.50309122]]
be